# This code builds a large dictionary of all lines with their verdicts

We need this to add the verdict to the lines that have been featurized

In [1]:
import pandas as pd

In this code, I clean the data from the problems if we have $ in the line. This problems makes the pandas to skip the line and we do not want that in the real code.

To clean it, we read the file line-by-line, split the line and check if the separator was used in the line (more splits than expected). If so, we join the line and skip the separator. 

Then we save the file.

In [15]:
with open('./gerrit_review_comments_dictionary_sentiment.csv', 'r', encoding='utf8') as fRaw:
    linesRaw = fRaw.readlines()

In [16]:
linesProcessed = []
for oneLine in linesRaw:
    lineEl = oneLine.split(';')
    nEl = len(lineEl)
    if nEl > 4:
        codeContent = ''.join(lineEl[3:])
        codeContent = codeContent.replace('$','_')
        strLine = '$'.join(lineEl[:3])
        strLine += '$' + codeContent
    else:
        strLine = '$'.join(lineEl)
    linesProcessed.append(strLine)

In [17]:
fOut= open('./gerrit_reviews_comments_dictionary_sentiment_cleaned.csv', 'w', encoding='utf8')

for oneLine in linesProcessed:
    fOut.write(oneLine + '\n')
    
fOut.close()

This code reads the csv file and creates a dictionary for the lines, which we can later on use when adding a verdict to a specific line. 

In [9]:
dfLines = pd.read_csv('./gerrit_reviews_comments_dictionary_sentiment_cleaned.csv', 
                      sep='$',
                     error_bad_lines=False,
                     warn_bad_lines=True,
                     index_col=False, 
                     encoding='utf8')

In [10]:
dfLines.head()

,filename,LOC,class_value
0,src/gromacs/applied_forces/densityfitting.cpp,"basic_mdspan<const float, dynamicExten...",0
1,src/gromacs/applied_forces/densityfitting.cpp,"basic_mdspan<const float, dynamicExten...",1
2,src/gromacs/applied_forces/densityfitting.cpp,"* Asserts that reading went okay, exi...",0
3,src/gromacs/applied_forces/densityfitting.cpp,"* Asserts that reading went okay, exi...",1
4,src/gromacs/applied_forces/densityfitting.cpp,= std::make_unique < Translate...,0


In [11]:
dictLines = dict()

for index, row in dfLines.iterrows():
    line = row['LOC']
    verdict_in_csv = row['class_value']
    if line in dictLines:
        verdict_in_dict = dictLines[line]
        if verdict_in_csv != verdict_in_dict:
            dictLines[line] = 1
    else:
        dictLines[line] = verdict_in_csv
        

In [12]:
print(len(dfLines))
print(len(dictLines))

149335
39123


In [14]:
dictLines.to_csv('./gerrit_comments_dictionary_cleaned.csv')

AttributeError: 'dict' object has no attribute 'to_csv'

After this script, the dictionary is clean, i.e. does not contain any duplicated lines nor inconsistent verdicts.

The file can be used by the merger which takes the featurized code and this file to create the training set.